<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/misc/read_from_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read from API
- Read data from API using requests and RDDs

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

# Get data from API - Vehicles

In [2]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [3]:
readFromAPI("https://api.carrismetropolitana.pt/municipalities").show()

+-----------+-------------+----+--------------------+------+---------+----------------+
|district_id|district_name|  id|                name|prefix|region_id|     region_name|
+-----------+-------------+----+--------------------+------+---------+----------------+
|         07|        Évora|0712|        Vendas Novas|    19|    PT187|Alentejo Central|
|         11|       Lisboa|1101|            Alenquer|    20|    PT16B|           Oeste|
|         11|       Lisboa|1102|   Arruda dos Vinhos|    20|    PT16B|           Oeste|
|         11|       Lisboa|1105|             Cascais|    05|    PT170|             AML|
|         11|       Lisboa|1106|              Lisboa|    06|    PT170|             AML|
|         11|       Lisboa|1107|              Loures|    07|    PT170|             AML|
|         11|       Lisboa|1109|               Mafra|    08|    PT170|             AML|
|         11|       Lisboa|1110|              Oeiras|    12|    PT170|             AML|
|         11|       Lisboa|1111|

# Vehicles

In [4]:
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

vehicles = readFromAPI("https://api.carrismetropolitana.pt/vehicles", vehicle_schema)
print(vehicles.count())
vehicles.show()

709
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|    127|         14_14213-71|   INCOMING_AT| 41|1127| 38.77032|   1211|-9.289865|  1211_0_2|  1211_0|            SCHEDULED|       14218|2.2222223| 170563|2025-06-13 14:17:17|1211_0_2_1500_152...|
|    108|20250613-64010192...| IN_TRANSIT_TO|44|12093|38.522484|   4410|-8.851563|  4410_0_2|  4410_0|            SCHEDULED|112590234560|      0.0| 160050|2025-06-13 14:17:21|4410_0_2|1300|144...|
|     42|20

# Routes

In [5]:
from pyspark.sql.functions import *
routes = readFromAPI("https://api.carrismetropolitana.pt/routes")
print(routes.count())
routes.filter(array_contains(col("localities"), "Colégio Militar") & array_contains(col("localities"), "Brandoa")).show(100, False)

933
+---------------+-------+----------+------+-------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------+--------------+--------------------+----------+----------+
|_corrupt_record|color  |facilities|id    |line_id|localities                                                                               |long_name                                                         |municipalities|patterns            |short_name|text_color|
+---------------+-------+----------+------+-------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------+--------------+--------------------+----------+----------+
|NULL           |#C61D23|[]        |1706_0|1706   |[Casal da Mira, Amadora, Brandoa, Alfornelos, Lisboa, Benfica, Colégio Militar]          |Colégio Militar (Metro) - UBBO via Brandoa            

# Questions

# Q1
- adjust ROUTE dataframe
  - use correct schema to get the data (StructField)
  - make sure _corrupt_record is removed from schema
- find the routes that pass through the localities "Brandoa" and "Colégio Militar"

### API - https://github.com/carrismetropolitana/api